# Train Recommender System
On this notebook we will use Pytorch to implement Recommender Systems using the following techniques:
* Matrix Factorisation
* Collaborative Filtering

#### Refereces
* https://blog.fastforwardlabs.com/2018/04/10/pytorch-for-recommenders-101.html
* https://medium.com/@iliazaitsev/how-to-implement-a-recommendation-system-with-deep-learning-and-pytorch-2d40476590f9
* https://towardsdatascience.com/recommender-systems-using-deep-learning-in-pytorch-from-scratch-f661b8f391d7
* http://hameddaily.blogspot.com/2016/12/simple-matrix-factorization-with.html?m=1

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import numpy as np

import models
from movie100k_dataset_np import MovieLensNumpy
import utils_train
print("PyTorch Version: ",torch.__version__)

# Jupyter Widgets
from IPython.display import clear_output, display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Hyperparameters
num_epochs = 50
batch_size = 1024 #20
base_lr = 1e-2
n_factors = 20
num_users = 944
num_movies = 1683

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
if device != 'cpu':
    num_gpu = torch.cuda.device_count()
    batch_size *= num_gpu
    base_lr *= num_gpu
    print('Number of GPUs Available:', num_gpu)

PyTorch Version:  1.1.0
Device: cuda:0
Number of GPUs Available: 8


#### Load Train/Validation data

In [2]:
train_dataset = MovieLensNumpy('train.npy')
val_dataset = MovieLensNumpy('val.npy')

# Create training and validation dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=16)
dataloaders_dict = {'train': train_loader, 'val': val_loader}

#### Initialize Tensorboard

In [3]:
# Default directory "runs"
writer = SummaryWriter()

#### Initialize Model

In [4]:
user_dummy = torch.randint(0, num_users, (10,))
movie_dummy = torch.randint(0, num_movies, (10,))
#recommender_model = models.MatrixFactorization(num_users, num_movies, n_factors=n_factors)
recommender_model = models.DenseNet(num_users, num_movies, n_factors=n_factors)

# Add on Tensorboard the Model Graph
writer.add_graph(recommender_model, [user_dummy, movie_dummy])

/mnt/anaconda3/lib/python3.7/site-packages/torch/onnx/symbolic.py:569: UserWarning: ONNX export squeeze with negative axis -1 might cause the onnx model to be incorrect. Negative axis is not supported in ONNX. Axis is converted to 1 based on input shape at export time. Passing an tensor of different rank in execution will be incorrect.
  "Passing an tensor of different rank in execution will be incorrect.")


#### Distribute model to multiple GPUs if available

In [5]:
if device != 'cpu':
    if num_gpu > 1:
        recommender_model = nn.DataParallel(recommender_model)
recommender_model = recommender_model.to(device)

#### Initialize Optimizer and Loss Function

In [6]:
loss_fn = torch.nn.MSELoss() 
# Can't use Adam/SparseAdam because of the Sparse Embedding
optimizer = torch.optim.Adam(recommender_model.parameters(),lr=base_lr)
sc_plt = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)    

#### Train

In [7]:
best_model = utils_train.train_model(recommender_model, dataloaders_dict, loss_fn, optimizer, sc_plt, writer, device, num_epochs=num_epochs)
torch.save(best_model.state_dict(), './best_model.pkl')

Epoch 0/49
----------
train Loss: 0.0006
val Loss: 0.0002
Epoch 1/49
----------
train Loss: 0.0002
val Loss: 0.0002
Epoch 2/49
----------
train Loss: 0.0002
val Loss: 0.0001
Epoch 3/49
----------
train Loss: 0.0002
val Loss: 0.0001
Epoch 4/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 5/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 6/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 7/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 8/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 9/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 10/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 11/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 12/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 13/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 14/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 15/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 16/49
----------
train Loss: 0.0001
val Loss: 0.0001
Epoch 1

#### Testing Model

In [8]:
best_model = recommender_model
@interact(idx=widgets.IntSlider(min=0,max=len(val_dataset)-1))
def explore_validation_dataset(idx):
    best_model.eval()
    sample = val_dataset[idx]
    user_id = torch.tensor(sample[0]).view(1,).to(device)
    movie_id = torch.tensor(sample[1]).view(1,).to(device)
    rating_label = sample[2]
    print('user_id:', user_id.item())
    print('movie_id:', movie_id.item())
    print('Rating Label:', rating_label)
    prediction = best_model([user_id, movie_id])    
    print('Rating Predicted:', prediction.item())

interactive(children=(IntSlider(value=0, description='idx', max=19997), Output()), _dom_classes=('widget-inter…